In [73]:
import pandas as pd
df=pd.read_csv('LOL 2021 Worlds Play-In Groups Stats.csv')
df.head(3)

,Team,Player,Opponent,Position,Champion,Kills,Deaths,Assists,Creep Score,Gold Earned,Champion Damage Share,Kill Participation,Wards Placed,Wards Destroyed,Ward Interactions,Dragons For,Dragons Against,Barons For,Barons Against,Result
0,UOL,Boss,GS,Top,Camille,4,5,3,188,11107,0.17,0.78,8,8,16,0,4,0,1,L
1,GS,Crazy,UOL,Top,Gwen,3,1,9,217,12201,0.20,0.52,10,7,17,4,0,1,0,W
2,UOL,Ahahacik,GS,Jungle,Trundle,2,4,5,156,9048,0.15,0.78,8,14,22,0,4,0,1,L


In [74]:
df.columns

Index(['Team', 'Player', 'Opponent', 'Position', 'Champion', 'Kills', 'Deaths',
       'Assists', 'Creep Score', 'Gold Earned', 'Champion Damage Share',
       'Kill Participation', 'Wards Placed', 'Wards Destroyed',
       'Ward Interactions', 'Dragons For', 'Dragons Against', 'Barons For',
       'Barons Against', 'Result'],
      dtype='object')

In [75]:
df1=df.drop(['Player', 'Opponent', 'Position', 'Champion', 'Creep Score', 'Gold Earned', 'Champion Damage Share','Kill Participation', 'Wards Placed', 'Wards Destroyed','Ward Interactions', 'Dragons For', 'Dragons Against', 'Barons For','Barons Against'],axis='columns')
df1.head(3)

,Team,Kills,Deaths,Assists,Result
0,UOL,4,5,3,L
1,GS,3,1,9,W
2,UOL,2,4,5,L


In [76]:
df1['Team'].unique()

array(['UOL', 'GS', 'DFM', 'C9', 'BYG', 'HLE', 'INF', 'LNG', 'PCE', 'RED'],
      dtype=object)

In [77]:
df1['Kills'].unique()

array([ 4,  3,  2,  5,  1,  9,  0,  8,  6,  7, 13, 10], dtype=int64)

In [78]:
df1['Deaths'].unique()

array([5, 1, 4, 3, 0, 2, 6, 7], dtype=int64)

In [79]:
df1['Assists'].unique()

array([ 3,  9,  5, 10,  4, 13, 16,  2,  1,  0,  6,  8,  7, 12, 11, 14, 15,
       19], dtype=int64)

In [80]:
df1['Result'].unique()

array(['L', 'W'], dtype=object)

In [81]:
dummies1=pd.get_dummies(df.Result)
merge1=pd.concat([df1,dummies1.drop('L',axis='columns')],axis='columns')
final1=merge1.drop(['Result'],axis='columns')
final1.head()

,Team,Kills,Deaths,Assists,W
0,UOL,4,5,3,False
1,GS,3,1,9,True
2,UOL,2,4,5,False
3,GS,5,4,10,True
4,UOL,1,3,4,False


In [82]:
dummies2=pd.get_dummies(df.Team)
merge2=pd.concat([final1,dummies2.drop('UOL',axis='columns')],axis='columns')
final=merge2.drop(['Team'],axis='columns')
final.head()

,Kills,Deaths,Assists,W,BYG,C9,DFM,GS,HLE,INF,LNG,PCE,RED
0,4,5,3,False,False,False,False,False,False,False,False,False,False
1,3,1,9,True,False,False,False,True,False,False,False,False,False
2,2,4,5,False,False,False,False,False,False,False,False,False,False
3,5,4,10,True,False,False,False,True,False,False,False,False,False
4,1,3,4,False,False,False,False,False,False,False,False,False,False


In [83]:
from sklearn import preprocessing
sx=preprocessing.MinMaxScaler()
sy=preprocessing.MinMaxScaler()
scaled_X=sx.fit_transform(final.drop('W',axis='columns'))
scaled_y=sy.fit_transform(final[['W']])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(scaled_X,scaled_y,test_size=0.2)

In [84]:
final.describe()

,Kills,Deaths,Assists
count,220.000000,220.000000,220.000000
mean,2.709091,2.718182,5.668182
std,2.579673,1.747179,3.888149
min,0.000000,0.000000,0.000000
25%,1.000000,1.000000,3.000000
50%,2.000000,3.000000,5.000000
75%,4.000000,4.000000,8.000000
max,13.000000,7.000000,19.000000


In [85]:
X=final.drop('W',axis='columns')
y=final['W']

In [86]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

In [87]:
def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'fit_intercept': [True, False]
            }
        },
        'logistic_regression': {
            'model': LogisticRegression(),
            'params': {
                'random_state': ['int',None],
                'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

In [93]:
import warnings
warnings.filterwarnings('ignore')

In [94]:
find_best_model_using_gridsearchcv(scaled_X,scaled_y)

,model,best_score,best_params
0,linear_regression,0.672111,{'fit_intercept': True}
1,logistic_regression,0.909091,"{'random_state': None, 'solver': 'lbfgs'}"
2,decision_tree,0.467799,"{'criterion': 'friedman_mse', 'splitter': 'best'}"


In [95]:
model=LogisticRegression(random_state= None, solver= 'lbfgs')
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.9318181818181818

In [96]:
X.columns

Index(['Kills', 'Deaths', 'Assists', 'BYG', 'C9', 'DFM', 'GS', 'HLE', 'INF',
       'LNG', 'PCE', 'RED'],
      dtype='object')

In [97]:
import numpy as np
def predict_match(kills,deaths,assists,team):    
    loc_index = np.where(X.columns==team)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = kills
    x[1] = deaths
    x[2] = assists
    if loc_index >= 0:
        x[loc_index] = 1

    return model.predict([x])[0]

In [98]:
predict_match(3,1,9,'GS')

1.0